In [2]:
import numpy as np
from keras import models
from keras import layers
import pandas as pd
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, make_scorer
# Set random seed
np.random.seed(42)

Using TensorFlow backend.


In [5]:
categorical_labels = [5,6,10,11,12,14,15,16,17,18,19,20]
other_labels = [1,2,3,4,7,8,9,13]
def split_data():
    """
    Reads data from file and splits it into X - input and y - output.
    :return: input(X) and output(y)
    """
    # Read data from a file and store it as a data frame
    X = pd.read_csv('ReducedWaterTraining.csv')
    #Remove column id & column Unnamed: 0 
    X.drop(['id','Unnamed: 0'], 1, inplace=True)
    #Replace all the empty strings with NaN  
    X.fillna(value=pd.np.nan, inplace=True)
    
    #Convert data frame to array
    X = np.array(X)
    #impute missing values
    X = impute_missing(X)
    #transform input
    X = transform_input(X)
    
    
    # Extract the output from a file
    y = pd.read_csv('training_set_labels.csv')
    #Drop id column
    y.drop(['id'], 1, inplace=True)
    #Transform categorical label
    y = y.apply(LabelEncoder().fit_transform)
    #Reshape
    y =  np.array(y).ravel()
    y.reshape(y.size, 1)
    
    return X, y

def transform_input(X):
    df = pd.DataFrame(X[:,categorical_labels])
    X[:,categorical_labels] = df.apply(LabelEncoder().fit_transform)
    onehotencoder = OneHotEncoder(categorical_features ='auto')
    for i in categorical_labels:
        onehotencoder = OneHotEncoder(categorical_features=[i])
        onehotencoder.fit_transform(X).toarray()
    sc = StandardScaler()
    X = sc.fit_transform(X)
    return X

def impute_missing(X):
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean').fit(X[:,other_labels])
    X[:,other_labels] = imp_mean.transform(X[:,other_labels])
    imp_most_frequent = SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit(X[:,categorical_labels])
    X[:,categorical_labels] = imp_most_frequent.transform(X[:,categorical_labels])
    return X
    

In [11]:
X,y = split_data()

/Users/gigisonnenschein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/gigisonnenschein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/Users/gigisonnenschein/anaconda3/lib/python3.6/site-packages/sklearn/preproce

/Users/gigisonnenschein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/gigisonnenschein/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/Users/gigisonnenschein/anaconda3/lib/python3.6/site-packages/sklearn/preproce

In [494]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print (classification_report(y_true, y_pred) )# print classification report
    return accuracy_score(y_true, y_pred) # return accuracy score

def create_cv():
    """
    Creates cross-validation splitting strategy.
    :return: cv
    """
    cv = StratifiedShuffleSplit(n_splits=5,train_size=0.7, random_state=42)
    return cv

def grid_search(clf, scoring, cv,param_grid, X, y):
    """
    Creates a model using GridSearchCV.
    :param clf: classifier
    :param parameters: classifier's parameters
    :param scoring: scoring parameters
    :param cv: cross-validation splitting strategy
    :param X: input
    :param y: output
    :return: model
    """
    grid_search_cv = GridSearchCV(clf,scoring=make_scorer(classification_report_with_accuracy_score),param_grid = param_grid, refit='accuracy', cv=cv).fit(X, y)
    model = grid_search_cv.best_estimator_
    return model


def create_network(optimizer='rmsprop'):
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=50, activation='softmax'))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=3, activation='softmax'))

    # Compile neural network
    network.compile(loss='categorical_crossentropy', # Cross-entropy
                    optimizer=optimizer, # Optimizer
                    metrics=['accuracy']) # Accuracy performance metric
    
    # Return compiled network
    return network



In [495]:
clf = KerasClassifier(build_fn=create_network, verbose=1)
cv = create_cv()
scoring = {'f1', 'roc_auc', 'accuracy'}
# Create hyperparameter space
epochs = [500]
batches = [100]
optimizers = ['adam']
# Create hyperparameter options
hyperparameters = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)
# Create grid search
grid = GridSearchCV(estimator=clf, param_grid=hyperparameters,cv = cv,refit='accuracy')
# Fit grid search
grid_result = grid.fit(X, y)
print(grid_result)

/Users/gigisonnenschein/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1788: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Epoch 1/500
41580/41580 [==============================] - 7s 160us/step - loss: 0.9462 - acc: 0.6061
Epoch 2/500
41580/41580 [==============================] - 2s 38us/step - loss: 0.8111 - acc: 0.6542
Epoch 3/500
41580/41580 [==============================] - 2s 42us/step - loss: 0.7605 - acc: 0.6842
Epoch 4/500
41580/41580 [==============================] - 2s 39us/step - loss: 0.7261 - acc: 0.7020
Epoch 5/500
41580/41580 [==============================] - 2s 38us/step - loss: 0.7013 - acc: 0.7112
Epoch 6/500
41580/41580 [==============================] - 2s 40us/step - loss: 0.6858 - acc: 0.7183
Epoch 7/500
41580/41580 [==============================] - 2s 39us/step - loss: 0.6764 - acc: 0.7222
Epoch 8/500
41580/41580 [==============================] - 2s 39us/step - loss: 0.6695 - acc: 0.7246
Epoch 9/500
41580/41580 [==============================] - 2s 39us/step - loss: 0.6638 - acc: 0.7263
Epoch 10/500
41580/41580 [==============================] - 2s 39us/step - loss: 0.6589 - 

41580/41580 [==============================] - 2s 41us/step - loss: 0.5802 - acc: 0.7601
Epoch 161/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5799 - acc: 0.7607
Epoch 162/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5800 - acc: 0.7602
Epoch 163/500
41580/41580 [==============================] - 2s 36us/step - loss: 0.5799 - acc: 0.7595
Epoch 164/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5799 - acc: 0.7601
Epoch 165/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5796 - acc: 0.7602
Epoch 166/500
41580/41580 [==============================] - 2s 36us/step - loss: 0.5796 - acc: 0.7608
Epoch 167/500
41580/41580 [==============================] - 2s 36us/step - loss: 0.5795 - acc: 0.7604
Epoch 168/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5793 - acc: 0.7608
Epoch 169/500
41580/41580 [==============================] - 1s 36us/step - loss: 0.579

41580/41580 [==============================] - 2s 37us/step - loss: 0.5697 - acc: 0.7654
Epoch 319/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5698 - acc: 0.7656
Epoch 320/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5698 - acc: 0.7654
Epoch 321/500
41580/41580 [==============================] - 1s 34us/step - loss: 0.5697 - acc: 0.7658
Epoch 322/500
41580/41580 [==============================] - 1s 36us/step - loss: 0.5696 - acc: 0.7657
Epoch 323/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5696 - acc: 0.7653
Epoch 324/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5694 - acc: 0.7652
Epoch 325/500
41580/41580 [==============================] - 1s 33us/step - loss: 0.5695 - acc: 0.7656
Epoch 326/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5695 - acc: 0.7654
Epoch 327/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.569

41580/41580 [==============================] - 2s 37us/step - loss: 0.5646 - acc: 0.7686
Epoch 477/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5646 - acc: 0.7687
Epoch 478/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5646 - acc: 0.7682
Epoch 479/500
41580/41580 [==============================] - 2s 36us/step - loss: 0.5647 - acc: 0.7678
Epoch 480/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5646 - acc: 0.7683
Epoch 481/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5647 - acc: 0.7688
Epoch 482/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5646 - acc: 0.7685
Epoch 483/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5645 - acc: 0.7678
Epoch 484/500
41580/41580 [==============================] - 1s 34us/step - loss: 0.5643 - acc: 0.7687
Epoch 485/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.564

...

41580/41580 [==============================] - 2s 37us/step - loss: 0.5637 - acc: 0.7690
Epoch 296/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5636 - acc: 0.7693
Epoch 297/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5638 - acc: 0.7695
Epoch 298/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5636 - acc: 0.7685
Epoch 299/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5637 - acc: 0.7696
Epoch 300/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5635 - acc: 0.7694
Epoch 301/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5636 - acc: 0.7685
Epoch 302/500
41580/41580 [==============================] - 2s 38us/step - loss: 0.5634 - acc: 0.7691
Epoch 303/500
41580/41580 [==============================] - 2s 36us/step - loss: 0.5634 - acc: 0.7683
Epoch 304/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.563

41580/41580 [==============================] - 2s 37us/step - loss: 0.5593 - acc: 0.7717
Epoch 454/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5592 - acc: 0.7715
Epoch 455/500
41580/41580 [==============================] - 1s 35us/step - loss: 0.5590 - acc: 0.7720
Epoch 456/500
41580/41580 [==============================] - 2s 38us/step - loss: 0.5590 - acc: 0.7718
Epoch 457/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5591 - acc: 0.7724
Epoch 458/500
41580/41580 [==============================] - 1s 36us/step - loss: 0.5590 - acc: 0.7732
Epoch 459/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5591 - acc: 0.7725
Epoch 460/500
41580/41580 [==============================] - 2s 37us/step - loss: 0.5591 - acc: 0.7720
Epoch 461/500
41580/41580 [==============================] - 2s 38us/step - loss: 0.5590 - acc: 0.7720
Epoch 462/500
41580/41580 [==============================] - 1s 34us/step - loss: 0.559

59400/59400 [==============================] - 2s 37us/step - loss: 0.5850 - acc: 0.7619
Epoch 112/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5848 - acc: 0.7623
Epoch 113/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5846 - acc: 0.7623
Epoch 114/500
59400/59400 [==============================] - 2s 38us/step - loss: 0.5846 - acc: 0.7625
Epoch 115/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5844 - acc: 0.7618
Epoch 116/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5842 - acc: 0.7618
Epoch 117/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5842 - acc: 0.7620
Epoch 118/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5840 - acc: 0.7627
Epoch 119/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5838 - acc: 0.7622
Epoch 120/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.583

59400/59400 [==============================] - 3s 43us/step - loss: 0.5719 - acc: 0.7668
Epoch 270/500
59400/59400 [==============================] - 2s 36us/step - loss: 0.5719 - acc: 0.7671
Epoch 271/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5719 - acc: 0.7671
Epoch 272/500
59400/59400 [==============================] - 2s 36us/step - loss: 0.5719 - acc: 0.7670
Epoch 273/500
59400/59400 [==============================] - 2s 36us/step - loss: 0.5718 - acc: 0.7670
Epoch 274/500
59400/59400 [==============================] - 2s 35us/step - loss: 0.5717 - acc: 0.7670
Epoch 275/500
59400/59400 [==============================] - 2s 36us/step - loss: 0.5716 - acc: 0.7671
Epoch 276/500
59400/59400 [==============================] - 2s 36us/step - loss: 0.5717 - acc: 0.7671
Epoch 277/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5715 - acc: 0.7674
Epoch 278/500
59400/59400 [==============================] - 2s 36us/step - loss: 0.571

59400/59400 [==============================] - 2s 37us/step - loss: 0.5657 - acc: 0.7702
Epoch 428/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5655 - acc: 0.7702
Epoch 429/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5654 - acc: 0.7698
Epoch 430/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5655 - acc: 0.7703
Epoch 431/500
59400/59400 [==============================] - 2s 38us/step - loss: 0.5654 - acc: 0.7700
Epoch 432/500
59400/59400 [==============================] - 2s 36us/step - loss: 0.5655 - acc: 0.7699
Epoch 433/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5654 - acc: 0.7699
Epoch 434/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5654 - acc: 0.7697
Epoch 435/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.5653 - acc: 0.7705
Epoch 436/500
59400/59400 [==============================] - 2s 37us/step - loss: 0.565

In [497]:
allscores=grid.cv_results_['mean_test_score']
print(allscores)

[0.75319865]
